In [114]:
import numpy as np
from numpy.linalg import norm
import ollama
import os
import json

link to video reference: https://www.youtube.com/watch?v=V1Mz8gMBDMo

In [115]:
def parse_file(filename):
    with open(filename, encoding='utf-8-sig') as f:
        paragraphs = []
        buffer = []
        for line in f.readlines():
            line = line.strip()
            if line:
                buffer.append(line)
            elif len(buffer):
                paragraphs.append(" ".join(buffer))
                buffer = []
        if len(buffer):
            paragraphs.append(" ".join(buffer))
    return paragraphs

def save_embeddings(filename, embeddings):
    # create directory if not exists
    if not os.path.exists("embeddings"):
        os.makedirs("embeddings")
    # dump embedding to json file
    with open(f"embeddings/{filename}.json", 'w') as f:
        json.dump(embeddings, f)

def load_embeddings(filename):
    # check if file exists
    if not os.path.exists(f"embeddings/{filename}.json"):
        return False
    with open(f"embeddings/{filename}.json", 'r') as f:
        return json.load(f)

def get_embeddings(filename, modelname, chunks):
    # check if embeddings are already computed
    if filename and ((embeddings := load_embeddings(filename)) is not False):
        return embeddings
    # compute embeddings
    embeddings = [
            ollama.embeddings(model=modelname, prompt=chunk)["embedding"] 
            for chunk in chunks
        ]
    # save embeddings
    if filename:
        save_embeddings(filename, embeddings)
    return embeddings

def find_most_similar(needle, haystack, sort=True):
    needle_norm = norm(needle)
    similarity_scores = [
        np.dot(needle, item) / (needle_norm * norm(item))
        for item in haystack
    ]
    if not sort:
        return zip(similarity_scores, range(len(haystack)))
    return sorted(zip(similarity_scores, range(len(haystack))), reverse=True)

def parse_response_llm(response):
    # split each line, then, take the text after the number and the dot (e.g. "1. ") or the number and the space (e.g. "1 ") or the number and the parenthesis (e.g. "1) ")
    result = []
    separators = [". ", ": ", ") ", ".", ":", ")"]
    for line_raw in response.split("\n"):
        line = line_raw.strip()
        if line:
            for sep in separators:
                if sep in line:
                    result.append(line.split(sep)[1])
                    break
    return result

In [221]:
filenames = ['pacte_novation_2.txt', 'jeux_olympiques_paris.txt']
modelname = "mistral"

### get the document embeddings

In [227]:
embeddings = []
paragraphs = []

for filename in filenames:
    paragraphs += parse_file(filename)
    embeddings += get_embeddings(filename, modelname, paragraphs)

### Prompt = query

In [228]:
prompt = "Comment travailler à Pacte Novation ?"
queries = [prompt]

### Query expansion

In [229]:
SYSTEM_PROMPT = """Vous êtes un assistant de modèle linguistique d'IA. Votre tâche consiste à générer cinq versions françaises différentes de la question donnée par l'utilisateur afin d'extraire les documents pertinents d'une base de données vectorielle. 
En générant des perspectives multiples sur la question de l'utilisateur, votre objectif est d'aider l'utilisateur à surmonter certaines limites de la recherche de similarité basée sur la distance.
l'utilisateur à surmonter certaines des limites de la recherche de similarité basée sur la distance. 
Fournissez ces questions alternatives séparées par des nouvelles lignes.
"""

response = ollama.chat(
    model=modelname,
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {"role": "user", "content": "Question originale : " + prompt},
    ],
)

queries = [prompt] + parse_response_llm(response["message"]["content"])

print("\n".join(queries))

Comment travailler à Pacte Novation ?
Comment réaliser une novation de contrat avec Pacte ?
Quels sont les étapes de processus pour effectuer une novation de contrat auprès de Pacte ?
En quelles circonstances et comment peut-on faire une modification d'un accord avec Pacte ?
Comment procéder à l'application d'une nouvelle disposition dans un contrat existant avec Pacte ?
Quels sont les modalités de modification d'un accord en vigueur avec Pacte : comment fonctionne la novation ?


### get the query embeddings

In [230]:
queries_embeddings = get_embeddings(None, modelname, queries)

### calculate the cosine similarity between the queries and the documents

In [231]:
total_score_chunks = []

for query, query_embedding in zip(queries, queries_embeddings):
    chunks = find_most_similar(query_embedding, embeddings, sort=False)
    total_score_chunks.append(chunks)

len_query = len(queries)

# addition of the scores of the chunks
total_score = [0] * len(embeddings)
for chunks in total_score_chunks:
    for score, index in chunks:
        total_score[index] += score / len_query

# sort the chunks by the total score
sorted_chunks = sorted(zip(total_score, range(len(embeddings))), reverse=True)

for score, index in sorted_chunks:
    print(f"Score: {score:.2f}")
    print(paragraphs[index])
    print()

Score: 0.67
Le 7 février 2018, un changement dans le calendrier est annoncé. Les JO, qui devaient initialement se dérouler du 2 au 18 août, sont avancés d'une semaine pour des raisons d'organisation. Ils se tiendront finalement du 26 juillet au 11 août 2024.

Score: 0.64
Il existe également un jeu concours où les participants ont un objectif de totaliser plus de 100 000 points en réalisant du sport quotidiennement avec un tirage final le 31 décembre 2023 pour attribuer les dossards.

Score: 0.63
En complément, Assigraph International, notre partenaire éditeur de logiciels, apporte une expertise reconnue dans les outils de CAO dédiés aux métiers de la schématique. Par ailleurs, notre organisme de formation, Adalog, se spécialise dans le langage Ada, renforçant ainsi nos compétences internes et externes.

Score: 0.63
Épreuves grand public Le comité d'organisation a souhaité créer des épreuves ouvertes au public qui se dérouleraient dans les mêmes conditions que le haut niveau. C'est déjà

In [232]:
# when score >= 0.65, we consider the document as relevant
similar_chunks = [(score, index) for score, index in sorted_chunks if score >= 0.62]
similar_chunks = similar_chunks[:5]

for score, index in similar_chunks:
    print(f"Score: {score:.2f}")
    print(paragraphs[index])
    print()

Score: 0.67
Le 7 février 2018, un changement dans le calendrier est annoncé. Les JO, qui devaient initialement se dérouler du 2 au 18 août, sont avancés d'une semaine pour des raisons d'organisation. Ils se tiendront finalement du 26 juillet au 11 août 2024.

Score: 0.64
Il existe également un jeu concours où les participants ont un objectif de totaliser plus de 100 000 points en réalisant du sport quotidiennement avec un tirage final le 31 décembre 2023 pour attribuer les dossards.

Score: 0.63
En complément, Assigraph International, notre partenaire éditeur de logiciels, apporte une expertise reconnue dans les outils de CAO dédiés aux métiers de la schématique. Par ailleurs, notre organisme de formation, Adalog, se spécialise dans le langage Ada, renforçant ainsi nos compétences internes et externes.

Score: 0.63
Épreuves grand public Le comité d'organisation a souhaité créer des épreuves ouvertes au public qui se dérouleraient dans les mêmes conditions que le haut niveau. C'est déjà